<a href="https://colab.research.google.com/github/likhitha-kadiyala/Sai-Likhitha_INFO5731_Spring2021/blob/main/in_class_exercise/In_class_exercise8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The eighth in-class-exercise (20 points in total, 3/30/2021)**

The data for this exercise is from the dataset you created from assignment three. Please perform answer the following questions based on your data:

## (1) (10 points) Write a python program to extract the sentiment related terms from the corpus. You may use python package such as polyglot or external lexicon resources in the question. Rank the sentiment related terms by frequency.

In [1]:
!pip install polyglot

     |████████████████████████████████| 133kB 19.1MB/s 
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52557 sha256=8115017d19e74035edde68fc7e8069d6449a173fa28332a4411ac63a5491bbaf
  Stored in directory: /root/.cache/pip/wheels/5e/91/ef/f1369fdc1203b0a9347d4b24f149b83a305f39ab047986d9da
Successfully built polyglot


In [2]:
!pip install pyicu

     |████████████████████████████████| 235kB 17.1MB/s 
  Created wheel for pyicu: filename=PyICU-2.6-cp37-cp37m-linux_x86_64.whl size=1306382 sha256=2b1c378340ca60d91e24b279e3af4c189ae94ff6622bccd33d1214f1c51ccca4
  Stored in directory: /root/.cache/pip/wheels/31/21/2f/1c91831e8a93537ab21f6b4b935781b681104635fdb0315791
Successfully built pyicu


In [3]:
!pip install pycld2

     |████████████████████████████████| 41.4MB 99kB/s 
  Created wheel for pycld2: filename=pycld2-0.41-cp37-cp37m-linux_x86_64.whl size=9834258 sha256=8650a0dc7dbe9189c38bc318863fc8f2fd00ce26c56926f494f8239527cf970f
  Stored in directory: /root/.cache/pip/wheels/c6/8f/e9/08a1a8932a490175bd140206cd86a3dbcfc70498100de11079
Successfully built pycld2


In [4]:
!pip install morfessor

In [5]:
%%bash
polyglot download sentiment2.en

[polyglot_data] Downloading package sentiment2.en to
[polyglot_data]     /root/polyglot_data...


In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
import pandas as pd
imdb_df = pd.read_csv('https://raw.githubusercontent.com/likhitha-kadiyala/Sai-Likhitha_INFO5731_Spring2021/main/Sentiment%20analysis.csv')
imdb_df = imdb_df.loc[:, ~imdb_df.columns.str.contains('^Unnamed')]

**Text Preprocessing**

In [9]:
from nltk.corpus import stopwords
from textblob import TextBlob
imdb_df['Reviews'] = imdb_df['Reviews'].apply(lambda x: " ".join(x.lower() for x in x.split()))
imdb_df['Reviews'] = imdb_df['Reviews'].str.replace('[^\w\s]','')
stop = stopwords.words('english')
imdb_df['Reviews'] = imdb_df['Reviews'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
imdb_df['Reviews'] = imdb_df['Reviews'].apply(lambda x: str(TextBlob(x).correct()))

**Extracting words using polyglot**

In [10]:
from polyglot.text import Text
senti_words = []
for value in imdb_df['Reviews']:
  text = Text(value)
  for a in text.words:
    if a.polarity != 0:
      senti_words.append(a)
senti_words

['brilliance',
 'grotesque',
 'hard',
 'mesmerizing',
 'tragic',
 'funny',
 'rollercoaster',
 'proper',
 'best',
 'isolated',
 'sorry',
 'encourage',
 'better',
 'respect',
 'like',
 'isolated',
 'masterpiece',
 'best',
 'best',
 'best',
 'willingly',
 'remarkable',
 'disturbing',
 'violent',
 'proving',
 'corruption',
 'rich',
 'talented',
 'believable',
 'shake',
 'bitter',
 'comforting',
 'perfect',
 'fearless',
 'stunning',
 'impossible',
 'dark',
 'joke',
 'stunned',
 'accomplished',
 'impossible',
 'exceeds',
 'dark',
 'crime',
 'like',
 'falling',
 'joke',
 'happy',
 'ignore',
 'complaints',
 'pernicious',
 'embarrassing',
 'better',
 'criminal',
 'dark',
 'joke',
 'dark',
 'fall',
 'love',
 'bad',
 'romantic',
 'scars',
 'amazing',
 'best',
 'breathtaking',
 'uncomfortable',
 'like',
 'horrible',
 'great',
 'inspiration',
 'honest',
 'violent',
 'disturbing',
 'violent',
 'controversy',
 'great',
 'best',
 'hate',
 'madness',
 'enjoy',
 'masterpiece',
 'amazing',
 'enjoy',
 'jo

**Ranking words using frequency**

In [11]:
from collections import Counter
freq = Counter(senti_words)
ranking_df = pd.DataFrame(list(freq.most_common()), columns=['Words', 'Frequencies'])
ranking_df.index = list(range(1, len(freq.most_common())+1))
ranking_df

,Words,Frequencies
1,joke,147
2,like,54
3,good,54
4,best,30
5,great,28
...,...,...
507,advantage,1
508,disappointing,1
509,creep,1
510,pleasantly,1


## (2) (10 points) Compare the performance of the following tools in sentiment identification: TextBlob (https://textblob.readthedocs.io/en/dev/), VADER (https://github.com/cjhutto/vaderSentiment), TFIDF-based Support Vector Machine (SVM) (Split your data into training and testing data). Take your own annotation as the standard answers. 

Reference code: https://towardsdatascience.com/fine-grained-sentiment-analysis-in-python-part-1-2697bb111ed4

In [12]:
def Encoder(senti_word):
  senti_encoded = 0
  if senti_word == 'Positive':
    senti_encoded = 1
  elif senti_word == 'Negative':
    senti_encoded = 2
  return senti_encoded

In [14]:
sentiment = []
for senti_word in imdb_df['Sentiment']:
  sentiment.append(Encoder(senti_word))

In [16]:
from textblob import TextBlob
textblob_senti = []
for line in imdb_df['Reviews']:
  polarity = TextBlob(line).sentiment.polarity
  if polarity > 0:
    textblob_senti.append(1)
  elif polarity < 0:
    textblob_senti.append(2)
  elif polarity == 0.0:
    textblob_senti.append(0)
textblob_df = pd.DataFrame(list(zip(imdb_df['Reviews'], sentiment, textblob_senti)), columns = ['Reviews', 'Actual', 'Predicted'])
textblob_df

,Reviews,Actual,Predicted
0,every movie comes truly makes impact joaquins ...,1,2
1,movie felt alone isolated truly relate underst...,0,0
2,truly masterpiece best hollywood film 2019 one...,1,1
3,joaquin phoenix gives tour de force performanc...,1,2
4,time moves anticipated like end falling short ...,1,1
...,...,...,...
101,one worst moves ever seen ok maybe hope surrou...,2,1
102,didn really know much movie hadn seen trailer ...,2,1
103,film directed produced poorly opportunity take...,2,2
104,now movie admit first heard joaquin phoenix go...,1,1


In [17]:
from sklearn.metrics import f1_score, accuracy_score
textblob_accuracy = accuracy_score(textblob_df['Actual'], textblob_df['Predicted'])*100
textblob_f1score = f1_score(textblob_df['Actual'], textblob_df['Predicted'], average = 'macro')
print("Text Blob Accuracy is {0} and F1 Score is {1}".format(textblob_accuracy, textblob_f1score))

Text Blob Accuracy is 50.943396226415096 and F1 Score is 0.38479768572989687


In [18]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [19]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()
vader_senti = []
for line in imdb_df['Reviews']:
  polarity = vader.polarity_scores(line)['compound']
  if polarity > 0:
    vader_senti.append(1)
  elif polarity < 0:
    vader_senti.append(2)
  elif polarity == 0.0:
    vader_senti.append(0)
vader_df = pd.DataFrame(list(zip(imdb_df['Reviews'], sentiment, vader_senti)), columns = ['Reviews', 'Actual', 'Predicted'])
vader_df

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


,Reviews,Actual,Predicted
0,every movie comes truly makes impact joaquins ...,1,1
1,movie felt alone isolated truly relate underst...,0,1
2,truly masterpiece best hollywood film 2019 one...,1,1
3,joaquin phoenix gives tour de force performanc...,1,1
4,time moves anticipated like end falling short ...,1,1
...,...,...,...
101,one worst moves ever seen ok maybe hope surrou...,2,1
102,didn really know much movie hadn seen trailer ...,2,1
103,film directed produced poorly opportunity take...,2,2
104,now movie admit first heard joaquin phoenix go...,1,1


In [20]:
from sklearn.metrics import f1_score, accuracy_score
vader_accuracy = accuracy_score(vader_df['Actual'], vader_df['Predicted'])*100
vader_f1score = f1_score(vader_df['Actual'], vader_df['Predicted'], average = 'macro')
print("Vader Accuracy is {0} and F1 Score is {1}".format(vader_accuracy, vader_f1score))

Vader Accuracy is 52.83018867924528 and F1 Score is 0.37699109893747434


In [21]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [22]:
TFid_vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)

In [25]:
Train_a, Test_a, Train_b, Test_b = train_test_split(imdb_df['Reviews'],imdb_df['Sentiment'],test_size=0.33)
Encoder = LabelEncoder()  
Train_b = Encoder.fit_transform(Train_b)
Test_b = Encoder.fit_transform(Test_b)
Train_a_vectors = TFid_vectorizer.fit_transform(Train_a)
Test_a_vectors = TFid_vectorizer.transform(Test_a)

In [27]:
from sklearn import svm
from sklearn.metrics import classification_report
svm_model = svm.SVC(kernel='linear')
svm_model.fit(Train_a_vectors, Train_b)
predicted = svm_model.predict(Test_a_vectors)
result = classification_report(Test_b, predicted, output_dict=True)
result

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'0': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 12},
 '1': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 5},
 '2': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 17},
 '3': {'f1-score': 0.06060606060606061,
  'precision': 0.03125,
  'recall': 1.0,
  'support': 1},
 'accuracy': 0.02857142857142857,
 'macro avg': {'f1-score': 0.015151515151515152,
  'precision': 0.0078125,
  'recall': 0.25,
  'support': 35},
 'weighted avg': {'f1-score': 0.0017316017316017316,
  'precision': 0.0008928571428571428,
  'recall': 0.02857142857142857,
  'support': 35}}

Text Blob: accuracy-50.94, F1 score-38.49
Vader: accuracy-52.83, F1 score-37.69
SVM: accuracy-2.857, F1 score-6.06

From all the above, we can see that accuracy is highest in Vader. F1 score is highest in Text Blob and secong hihest in Vader. By considering all these we can say that Vader is better than text blob and SVM.